In [136]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
import pandas as pd
import numpy as np
import matplotlib as pltb

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [138]:
df = pd.read_csv('dataset-alcohol.csv')
df.describe(include='all')

,age_yrs,height_cm,weight_kg,duration_mins,elapsed_mins,sex,last_meal,units,over_limit
count,76.000000,76.000000,76.000000,76,76,76,76,76.000000,76
unique,NaN,NaN,NaN,16,9,2,5,NaN,2
top,NaN,NaN,NaN,120,10,Male,Full,NaN,No
freq,NaN,NaN,NaN,15,61,60,33,NaN,46
mean,22.657895,176.644737,71.486842,NaN,NaN,NaN,NaN,8.632895,NaN
std,5.627439,8.453329,11.474602,NaN,NaN,NaN,NaN,5.775567,NaN
min,18.000000,157.000000,47.000000,NaN,NaN,NaN,NaN,0.000000,NaN
25%,19.000000,172.000000,63.000000,NaN,NaN,NaN,NaN,4.275000,NaN
50%,21.000000,177.000000,72.000000,NaN,NaN,NaN,NaN,8.400000,NaN
75%,23.000000,182.000000,79.000000,NaN,NaN,NaN,NaN,12.100000,NaN


In [150]:
df.shape

(73, 13)

In [140]:
df.dtypes

age_yrs            int64
height_cm          int64
weight_kg          int64
duration_mins     object
elapsed_mins      object
sex               object
last_meal         object
units            float64
over_limit        object
dtype: object

In [141]:
df['duration_mins'].unique()

array(['?', '60', '90', '120', '150', '240', '30', '270', '180', '330',
       '435', '325', '300', '360', '315', '5'], dtype=object)

In [142]:
df[df['duration_mins'] == '?']

,age_yrs,height_cm,weight_kg,duration_mins,elapsed_mins,sex,last_meal,units,over_limit
0,40,170,75,?,?,Male,Lunch,0.0,No
31,18,177,57,?,?,Female,Full,0.0,No


In [143]:
df['elapsed_mins'].unique()

array(['?', '10', '30', '5', '60', '40', '120', '15', '180'], dtype=object)

In [144]:
df[df['elapsed_mins'] == '?']

,age_yrs,height_cm,weight_kg,duration_mins,elapsed_mins,sex,last_meal,units,over_limit
0,40,170,75,?,?,Male,Lunch,0.0,No
31,18,177,57,?,?,Female,Full,0.0,No


In [145]:
df['last_meal'].unique()

array(['Lunch', 'Full', 'Snack', '?', 'None'], dtype=object)

In [146]:
df[df['last_meal'] == '?']

,age_yrs,height_cm,weight_kg,duration_mins,elapsed_mins,sex,last_meal,units,over_limit
8,23,170,66,60,30,Male,?,4.5,No


age_yrs            int64
height_cm          int64
weight_kg          int64
duration_mins     object
elapsed_mins      object
sex               object
last_meal         object
units            float64
over_limit        object
dtype: object
    
  
Delete both of these columns?
- Duration_mins, select the mean
- elapsed_mins, select mean

- sex is fine
- last meal needs a mode.

In [147]:
#Make a new copy with the deleted values for which there is some shit.

#df_delete = (df[(df['duration_mins'] != '?')]).copy()
#df_delete.reset_index(drop = True, inplace = True)

df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df_delete.describe(include='all')
df_delete['duration_mins'] = df_delete['duration_mins'].astype(int)
df_delete['elapsed_mins'] = df_delete['elapsed_mins'].astype(int)



#One hot encode...

one_hot = pd.get_dummies(df['sex'], 'sex', '_')
df.drop('sex', axis=1, inplace=True)
df = pd.concat([df, one_hot], axis=1)

one_hot = pd.get_dummies(df['last_meal'], 'last_meal', '_')
df.drop('last_meal', axis=1, inplace=True)
df = pd.concat([df, one_hot], axis=1)

#Some replacement for the over the limit thang.

df.replace({'over_limit' : {'Yes' : 1, 'No' : 0}}, inplace=True)

#LabelEncoder - Same as the replace but for more than two values.

In [151]:
m, n = df.shape
df_delete.describe(include='all')

,age_yrs,height_cm,weight_kg,duration_mins,elapsed_mins,units,over_limit,sex_Female,sex_Male,last_meal_?,last_meal_Full,last_meal_Lunch,last_meal_None,last_meal_Snack
count,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000
mean,22.486486,176.729730,71.635135,156.081081,16.554054,8.866216,0.405405,0.202703,0.797297,0.013514,0.432432,0.283784,0.148649,0.121622
std,5.299892,8.532448,11.498153,100.537154,25.413373,5.672217,0.494322,0.404757,0.404757,0.116248,0.498795,0.453911,0.358170,0.329079
min,18.000000,157.000000,47.000000,5.000000,5.000000,1.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,172.000000,63.250000,67.500000,10.000000,4.650000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,21.000000,177.000000,72.000000,120.000000,10.000000,9.050000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,23.000000,182.000000,79.000000,240.000000,10.000000,12.300000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,45.000000,200.000000,101.000000,435.000000,180.000000,31.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [152]:
df.columns

Index(['age_yrs', 'height_cm', 'weight_kg', 'duration_mins', 'elapsed_mins',
       'units', 'over_limit', 'sex_Female', 'sex_Male', 'last_meal_Full',
       'last_meal_Lunch', 'last_meal_None', 'last_meal_Snack'],
      dtype='object')

In [149]:
df = df_delete.take(np.random.permutation(m))
df.reset_index(drop=True, inplace=True)

X = df[['age_yrs', 'height_cm', 'weight_kg', 'duration_mins', 'elapsed_mins',  \
        'units', 'sex_Female', 'sex_Male', 'last_meal_Full', 'last_meal_Lunch',\
        'last_meal_None', 'last_meal_Snack']].values
y = df['over_limit'].values


#--------------FINISH ON MONDAY-------------------#


ols = Pipeline([('standardize', StandardScaler()),
                ('estimator', LogisticRegression())])
 
 
lassocv = Pipeline([('standardize', StandardScaler()),
                    ('estimator', LassoCV(cv=10))])
 
 
ridgecv = Pipeline([('standardize', StandardScaler()),
                    ('estimator', RidgeCV(cv=10))])
 
 
knn = Pipeline([('standardize', StandardScaler()),
                ('estimator', KNeighborsClassifier(weights = 'distance'))])
 

#Remeber that it's estimator__n_neighbors
knn_hyperparameters = {'estimator__n_neighbors' : [1, 2, 3, 4, 5, 6, 7,8, 9, 10]}
knn_gs = GridSearchCV(knn, knn_hyperparameters, scoring = 'accuracy', cv = 10)